## Code Agent Cookbook with Seed1.8

This cookbook implements a **Code Agent** with **medium reasoning effort** using **Seed1.8**. The Code Agent extends the model via function tools, allowing the model to **automatically select and call local functions** (e.g., compute, search, HTTP API access), then **merge tool outputs** in a second completion to form a closed loop.


## Features

- Tool Use: Register function tools with JSON schemas; the model plans and calls them autonomously.
- Closed Loop: Feed tool results back to the model for final answers.
- Single/Multi-turn: Support single-turn and multi-turn inference patterns.
- Error Handling: Minimal retries, fallbacks, and safe defaults for robust execution.


### 0. Prerequisites
- API Key: [Obtain an API key](https://console.volcengine.com/ark/region:ark+cn-beijing/apiKey) for the Seed1.8 model.
- Model Endpoint: Ensure the base URL of the Ark API.
- Activate the Model: Activate the Seed1.8 model in [the Ark Console](https://console.volcengine.com/ark/region:ark+cn-beijing/openManagement).
- Install Dependencies: Install required Python packages.


In [ ]:
pip install -r ../requirements.txt

In [ ]:
# Copyright 2025 Bytedance Ltd. and/or its affiliates.
# SPDX-License-Identifier: Apache-2.0

# Please set the API key here
import os

os.environ['ARK_API_KEY']  = 'your_ark_api_key'
os.environ['ARK_MODEL_ENDPOINT'] = "doubao-seed-1-8-251215"
base_url = "https://ark.cn-beijing.volces.com/api/v3"

### 1. Tool Schemas

Define minimal, safe function tools. The model can call these to accomplish tasks, then you feed the tool outputs back for final completion.


In [10]:
# tool schemas
# below are some example tools from OpenHands framework, see: https://github.com/OpenHands/OpenHands
tool_schemas = [
    {
        "type": "function",
        "function": {
            "name": "str_replace_editor",
            "description": "Custom editing tool for viewing, creating and editing files in plain-text format\n    * State is persistent across command calls and discussions with the user\n    * If `path` is a file, `view` displays the result of applying `cat -n`. If `path` is a directory, `view` lists non-hidden files and directories up to 2 levels deep\n    * The `create` command cannot be used if the specified `path` already exists as a file\n    * If a `command` generates a long output, it will be truncated and marked with `<response clipped>`\n    * The `undo_edit` command will revert the last edit made to the file at `path`\n\n    Notes for using the `str_replace` command:\n    * The `old_str` parameter should match EXACTLY one or more consecutive lines from the original file. Be mindful of whitespaces!\n    * If the `old_str` parameter is not unique in the file, the replacement will not be performed. Make sure to include enough context in `old_str` to make it unique\n    * The `new_str` parameter should contain the edited lines that should replace the `old_str`\n    ",
            "parameters": {
                "type": "object",
                "properties": {
                    "command": {
                        "description": "The commands to run. Allowed options are: `view`, `create`, `str_replace`, `insert`, `undo_edit`.",
                        "enum": ["view", "create", "str_replace", "insert", "undo_edit"],
                        "type": "string"
                    },
                    "path": {
                        "description": "Absolute path to file or directory, e.g. `/workspace/file.py` or `/workspace`.",
                        "type": "string"
                    },
                    "file_text": {
                        "description": "Required parameter of `create` command, with the content of the file to be created.",
                        "type": "string"
                    },
                    "old_str": {
                        "description": "Required parameter of `str_replace` command containing the string in `path` to replace.",
                        "type": "string"
                    },
                    "new_str": {
                        "description": "Optional parameter of `str_replace` command containing the new string (if not given, no string will be added). Required parameter of `insert` command containing the string to insert.",
                        "type": "string"
                    },
                    "insert_line": {
                        "description": "Required parameter of `insert` command. The `new_str` will be inserted AFTER the line `insert_line` of `path`.",
                        "type": "integer"
                    },
                    "view_range": {
                        "description": "Optional parameter of `view` command when `path` points to a file. If none is given, the full file is shown. If provided, the file will be shown in the indicated line number range, e.g. [11, 12] will show lines 11 and 12. Indexing at 1 to start. Setting `[start_line, -1]` shows all lines from `start_line` to the end of the file.",
                        "items": {
                            "type": "integer"
                        },
                        "type": "array"
                    }
                },
                "required": ["command", "path"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "execute_bash",
            "description": "Execute a bash command in the terminal.\n    * Long running commands: For commands that may run indefinitely, it should be run in the background and the output should be redirected to a file, e.g. command = `python3 app.py > server.log 2>&1 &`.\n    * One command at a time: You can only execute one bash command at a time. If you need to run multiple commands sequentially, you can use `&&` or `;` to chain them together.\n    ",
            "parameters": {
                "type": "object",
                "properties": {
                    "command": {
                        "type": "string",
                        "description": "The bash command to execute. Can be empty string to view additional logs when previous exit code is `-1`. Can be `C-c` (Ctrl+C) to interrupt the currently running process. Note: You can only execute one bash command at a time. If you need to run multiple commands sequentially, you can use `&&` or `;` to chain them together."
                    }
                },
                "required": ["command"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "finish",
            "description": "Finish the interaction when the task is complete OR if the assistant cannot proceed further with the task.",
            "parameters": {
                "type": "object",
                "properties": {},
                "required": []
            }
        }
    }
]

### 2. Code Setup

- **Prepare Inputs**:
   Define system prompts and user instructions; construct messages in Ark Chat format.

- **Model API Configuration**:
   Set `api_key`, `base_url`, and `model` endpoint; replace placeholders with your Seed-1.8 credentials.

In [14]:

import json
import os
import requests
# inputs prepare

def inference(messages, tool_schemas=[]):
    """Single-turn inference against Ark Chat API."""

    headers = {
        'Authorization': f'Bearer {api_key}',
        'Content-Type': 'application/json'
    }

    data = {
        "model": model,
        "messages": messages,
        "max_tokens": 4096,
        "top_p": 0.9,
        "tools": tool_schemas,
        "temperature": 0.0,
        "reasoning_effort": "medium"
    }
    
    response = requests.post(base_url, headers=headers, json=data, timeout=30)
    try:
        print(f"Response: {response.json()}")
    except:
        print("Error fetching response.json:")
        print(response)
        
    if response.status_code == 200:
        return response.json()
    else:
        print(f"Request failed with status code {response.status_code}")
        print(response.json())
        return {
            "error": f"Request failed with status code {response.status_code}",
            "details": response.text
        }
        
base_url = "https://ark.cn-beijing.volces.com/api/v3/chat/completions"
api_key = os.getenv('ARK_API_KEY')
model = os.getenv('ARK_MODEL_ENDPOINT')

messages = [
    {
        "role": "system",
        "content": "You are an agent designed to accomplish tasks.\n\nYou are a coding agent designed to accomplish software engineering tasks. Your goal is to take the user's query and to write high quality, well-designed code that fulfills the request. If a user provides a path, do not assume it is relative to the current working directory. Instead, you should explore the file system to find the file before working on it. When configuring git credentials, use \"openhands\" as the user.name and \"openhands@all-hands.dev\" as the user.email by default, unless explicitly instructed otherwise. By default, do not write comments in the code unless it is necessary to describe non-obvious behavior. Generate safe code and secure code. Do not write code that will create memory leaks or security vulnerabilities."
    },
    {
        "role": "user",
        "content": "Write a file named \"test.py\" with the content \"print('hello world')\""
    }
]

### 3. Model Inference

In [15]:
# Model inference
response = inference(messages, tool_schemas)

# Output model prediction results
print("\n" + "=" * 40)
print("🌟 🌟 🌟  **Model Prediction**  🌟 🌟 🌟")
print("=" * 40)
print(json.dumps(response, indent=4))  # Format JSON output

# Parse tool calls
actions = []
print("\n" + "=" * 40)
print("🔧 **Tool Calls** 🔧")
print("=" * 40)
for tool_call in (response.get("choices") or [{}])[0].get("message", {}).get("tool_calls", []) or []:
    print("\n📌 Tool Call:")
    print(json.dumps(tool_call, indent=4, ensure_ascii=False))  # Format JSON output
    function_name = tool_call.get("function", {}).get("name")
    arguments_raw = tool_call.get("function", {}).get("arguments", "{}")
    try:
        function_arguments = json.loads(arguments_raw)
    except Exception:
        function_arguments = {}
    actions.append(
        {
            "action_type": function_name,
            "action_inputs": function_arguments
        }
    )

Response: {'choices': [{'finish_reason': 'tool_calls', 'index': 0, 'logprobs': None, 'message': {'content': '', 'reasoning_content': 'Got it, let\'s see. The user wants me to create a file called test.py with the content print(\'hello world\'). First, I need to use the str_replace_editor tool with the create command. The path should be /workspace/test.py (since that\'s the default workspace), and the file_text is exactly the print statement they specified. Let me make sure the parameters are correct: command is create, path is /workspace/test.py, file_text is "print(\'hello world\')". Yeah, that should do it.', 'role': 'assistant', 'tool_calls': [{'function': {'arguments': '{"command":"create","path":"/workspace/test.py","file_text":"print(\'hello world\')"}', 'name': 'str_replace_editor'}, 'id': 'call_4y9qac8xc5zrqmi5s2v4aek8', 'type': 'function'}]}}], 'created': 1765438890, 'id': '0217654388866596cacdbb323ed1672e3f8caec678da11178f9c1', 'model': 'doubao-seed-1-8-preview-251115', 'serv

### 4. Multi-Turn Inference

For complex tasks, the agent needs to perform a series of actions, such as reading a file, editing its content, and then running a test. Multi-turn inference enables this by repeatedly calling the model with updated context (including tool results) until the task is complete. This example demonstrates a basic loop where the agent can make multiple tool calls across several steps, using a simple loop without memory condensation for demonstration.


In [16]:
# Multi-turn Model inference
def simple_multi_turn_inference(instruction, max_steps=10):
    """
    Perform simplified multi-turn inference without memory condensing.
    
    Args:
        instruction (str): The user's query or task instruction.
        max_steps (int): Maximum number of steps allowed for the inference process.
    
    Returns:
        str: The final answer provided by the agent.
    """
    # Initialize the conversation with system and user messages
    messages = [
        {
            "role": "system",
            "content": (
                "You are an agent designed to accomplish tasks.\n\nYou are a coding agent designed to accomplish software engineering tasks. Your goal is to take the user's query and to write high quality, well-designed code that fulfills the request. If a user provides a path, do not assume it is relative to the current working directory. Instead, you should explore the file system to find the file before working on it. When configuring git credentials, use \"openhands\" as the user.name and \"openhands@all-hands.dev\" as the user.email by default, unless explicitly instructed otherwise. By default, do not write comments in the code unless it is necessary to describe non-obvious behavior. Generate safe code and secure code. Do not write code that will create memory leaks or security vulnerabilities."
            )
        },
        {
            "role": "user",
            "content": instruction
        }
    ]
    
    # Initialize variables
    finished = False
    cur_step = 0
    answer = None
    
    while not finished and cur_step < max_steps:
        print(f"\n{'=' * 40}")
        print(f"🔄 Step {cur_step + 1}/{max_steps}")
        print(f"{'=' * 40}")
        
        # Perform single-turn inference
        response = inference(messages, tool_schemas)
        messages.append(response["choices"][0]["message"])  # Append the model's response to the conversation history
        
        # Check the finish reason to determine the next action
        finish_reason = response["choices"][0]["finish_reason"]
        if finish_reason == "stop":
            answer = response["choices"][0]["message"].get("content")
            finished = True
            print("✅ Agent Finished.")
        elif finish_reason == "tool_calls":
            # The model has requested tool usage
            tool_calls = response["choices"][0]["message"]["tool_calls"]
            
            print("\n" + "=" * 40)
            print("🔧 **Tool Calls** 🔧")
            print("=" * 40)
            
            for tool_call in tool_calls:
                tool_name = tool_call["function"]["name"]
                tool_args = json.loads(tool_call["function"]["arguments"])
                tool_call_id = tool_call["id"]
                
                print(f"\n📌 Tool Call: {tool_name}")
                print(json.dumps(tool_call, indent=4, ensure_ascii=False))
                
                # NOTE: In production, replace this mock response with actual tool execution in your agent environment
                tool_response = f"Mock execution result for {tool_name} with args: THIS IS A MOCK RESPONSE"
                
                messages.append({
                    "role": "tool",
                    "content": tool_response,
                    "tool_call_id": tool_call_id
                })
                print(f"🔧 Tool response added to conversation")
        else:
            # Handle unknown finish reasons
            print(f"⚠️ Unknown finish_reason: {finish_reason}")
            break
        
        # Print the response for debugging
        print(f"📝 Response: {json.dumps(response, indent=4, ensure_ascii=False)}")
        cur_step += 1
    
    # Final output
    if finished:
        print(f"\n🎉 Task completed in {cur_step} steps!")
        print(f"✅ Final Answer: {answer}")
    else:
        print(f"\n❌ Failed to complete the task within {cur_step} steps.")
    
    return answer


# Example usage
answer = simple_multi_turn_inference(
    instruction="Create a file called hello.txt in the current directory. Write \"Hello, world!\" to it.",
    max_steps=3
)


🔄 Step 1/3
Response: {'choices': [{'finish_reason': 'tool_calls', 'index': 0, 'logprobs': None, 'message': {'content': '', 'reasoning_content': 'Got it, let\'s see. The user wants me to create hello.txt in the current directory with "Hello, world!" in it. First, I should use the str_replace_editor tool with create command. The path should be ./hello.txt? Wait no, wait the parameter says absolute path? Wait no, wait let me check the function definition. Oh, the path can be absolute or maybe relative? Wait the example shows /workspace/file.py, but maybe if I use ./hello.txt that\'s okay? Wait no, wait let\'s think: the current directory—wait maybe I should use the execute_bash to check current directory first? Wait no, the user said "current directory", so let\'s just create it with str_replace_editor create command, path ./hello.txt, file_text "Hello, world!". Yeah that should work. Let me structure the function call correctly.', 'role': 'assistant', 'tool_calls': [{'function': {'argum

### 5. Best Practices

- Prefer task-oriented tools with fixed interfaces and structured outputs.
- Add timeouts and safe defaults for external calls; avoid destructive actions.
- Use structured outputs (JSON or strictly formatted text) for stable second completion.
- For long tasks, use iterative loops with tool calls and partial outputs; enable streaming if needed.
